In [ ]:
import sys
sys.path.insert(0, r'D:\Projects\aaa')

import pandas as pd
import matplotlib.pyplot as plt

import random
import numpy as np
random.seed(42)
np.random.seed() 

from src.utils.utils import get_files_in_from_directory

In [ ]:
allx = r'D:\Projects\aaa\results\rq4_results\features.csv'
npm = r'D:\Projects\aaa\results\rq4_results\features_npm.csv'
pypi = r'D:\Projects\aaa\results\rq4_results\features_pypi.csv'
mvn = r'D:\Projects\aaa\results\rq4_results\features_mvn.csv'

df_all = pd.read_csv(npm)

In [ ]:
# Sanitize time_to_next_merge where nan is a valid value
max_merge = df_all['time_to_next_merge'].max() * 10
df_all.loc[df_all['time_to_next_merge'].isna(),'time_to_next_merge'] = max_merge

df_all.fillna(0, inplace=True)
df_all.fillna(0, axis=1, inplace=True)

In [ ]:
print(df_all.shape[0])
print((df_all['label_security_related']==True).sum())

In [ ]:
_byrepo = df_all.groupby('label_repo_full_name')
repos = df_all['label_repo_full_name'].unique()
train_repos = random.sample(list(repos), int(0.9*len(repos)))
df = df_all[df_all.apply(lambda x: x['label_repo_full_name'] in train_repos, axis=1)]
eval_df = df_all[df_all.apply(lambda x: x['label_repo_full_name'] not in train_repos, axis=1)]

print(df.shape, (df['label_security_related']==True).sum())
print(eval_df.shape, (eval_df['label_security_related']==True).sum())


In [ ]:
# nan_ratio = {col:0 for col in df}

# for col in df:
#     nan_counts = df[col].isna().sum()
#     nan_ratio[col] = nan_counts/df.shape[0]

# nan_ratio = {k: v for k,v in sorted(nan_ratio.items(), key=lambda kv: -kv[1])}
# for k,v in nan_ratio.item():
#     if v > 0.1:
#         print(nan_ratio)


In [ ]:
broken_features = [
    'commits_to_next_merge',
    'commits_since_last_merge',
    'commits_to_next_merge',
    'commits_since_last_merge',
]

In [ ]:
no_variance_features = [
    'methods_with_attack_count_avg',
    'methods_with_corrupt_count_avg',
    'methods_with_corrupt_count_max',
    'methods_with_crash_count_avg',
    'methods_with_crash_count_max',
    'methods_with_deadlock_count_avg',
    'methods_with_deadlock_count_max',
    'methods_with_exploit_count_avg',
    'methods_with_exploit_count_max',
    'methods_with_leak_count_avg',
    'methods_with_malicious_count_avg',
    'methods_with_malicious_count_max',
    'methods_with_segfault_count_avg',
    'methods_with_segfault_count_max',
    'methods_with_sensit_count_avg',
    'methods_with_vulnerab_count_avg',
    'methods_with_vulnerab_count_max',
    'segfault_in_file_content',
    'segfault_in_message',
    'segfault_in_patch',
]

In [ ]:
highly_correlated_features = [
    'added_lines_count_avg',
    'added_lines_ratio_avg',

    'secur_in_title',
    'vulnerab_in_title',
    'exploit_in_title',
    'certificat_in_title',
    'authent_in_title',
    'leak_in_title',
    'sensit_in_title',
    'crash_in_title',
    'attack_in_title',
    'deadlock_in_title',
    'segfault_in_title',
    'malicious_in_title',
    'corrupt_in_title',

    'committed_by_bot',

    'avg_method_complexity_avg',
    'file_complexity_max',
    'file_token_count_max',
    'file_changed_method_count_max',
    'file_nloc_max',
    'max_method_token_count_avg',
    'total_methods_count_max',

    
    'changes_to_file_in_prev_50_commits_avg',
    'changes_to_file_in_next_50_commits_avg',
    ]
    

In [ ]:

df_selected = df
df_selected = df_selected[df_selected.columns.difference(broken_features)]
df_selected = df_selected[df_selected.columns.difference(no_variance_features)]
df_selected = df_selected[df_selected.columns.difference(highly_correlated_features)]

X = df_selected[df_selected.columns.difference(['label_repo_full_name', 'label_sha', 'label_commit_date', 'label_security_related'])]
y = df_selected['label_security_related']


In [ ]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.001)
sel.fit(X)
sup = sel.get_support()
i = -1
for x in X:
       i+=1
       if not sup[i]:
              print(f'reject \'{x}\',')


In [ ]:
correlation_matrix = X.corr()
corr = correlation_matrix.values
column_names = correlation_matrix.columns

for i in range(len(column_names)):
    for j in range(i+1, len(column_names)):
        if abs(corr[i,j])> 0.75:
            print('reject', column_names[i], ' ', column_names[j], ' ', corr[i,j])

In [ ]:
print('Features selected')

In [ ]:
import numpy as np

# manual rebalancing
oversample_ratio = 2
undersample_ratio = 2

x_positive = X.where(y).dropna()
x_negative = X.where(y==False).dropna()

x_positive = pd.concat([x_positive for i in range(oversample_ratio)])
x_negative = x_negative.sample(len(x_positive)*undersample_ratio)

X_resampled = pd.concat([x_negative, x_positive])
y_resampled = np.array([False]*len(x_negative) + [True]*len(x_positive))


In [ ]:
print(x_positive.shape, x_negative.shape, X_resampled.shape)

In [ ]:
# statistical rebalancing

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from sklearn.preprocessing import RobustScaler


# sampler = SMOTE()
# X_resampled, y_resampled = sampler.fit_resample(X_resampled, y_resampled)

scaler = RobustScaler().fit(X_resampled)
X_resampled = scaler.transform(X_resampled)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import recall_score, precision_score

VERBOSE_LVL = 10

In [ ]:
# Train selector & RF
    
model = RandomForestClassifier(
    n_estimators = 100,
    random_state=42)

selector = RFECV(model, 
    step=1, 
    cv=5,
    min_features_to_select=40,
    scoring = 'f1',
    n_jobs=-1,
    verbose=VERBOSE_LVL)

selector.fit(X_resampled, y_resampled)

print('max f1:', max(selector.cv_results_['mean_test_score']))

In [ ]:
#dict_keys(['mean_test_score', 'std_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score'])
# print(selector.cv_results_['mean_test_score'])

In [ ]:
colums = list(X.columns)

print('Selected features')
for i in range(len(selector.support_)):
    if selector.support_[i]:
        print(colums[i])

In [ ]:
import numpy as np

df_eval_selected = eval_df
df_eval_selected = df_eval_selected[df_eval_selected.columns.difference(broken_features)]
df_eval_selected = df_eval_selected[df_eval_selected.columns.difference(no_variance_features)]
df_eval_selected = df_eval_selected[df_eval_selected.columns.difference(highly_correlated_features)]

eval_X = df_eval_selected[df_eval_selected.columns.difference(['label_repo_full_name', 'label_sha', 'label_commit_date', 'label_security_related'])]
eval_y = df_eval_selected['label_security_related']


eval_x_positive = eval_X.where(eval_y).dropna()
eval_x_negative = eval_X.where(eval_y==False).dropna()
print(eval_x_positive.shape, eval_x_negative.shape)

eval_x_negative_sampled =  eval_x_negative.sample(len(eval_x_positive)*undersample_ratio)

eval_X_balanced = pd.concat([eval_x_negative_sampled, eval_x_positive])
eval_y_balanced = np.array([False]*len(eval_x_negative_sampled) + [True]*len(eval_x_positive))


In [ ]:
# eval RF selector

eval_scaled = scaler.transform(eval_X)
eval_selected_scaled = selector.transform(eval_scaled)
X_resampled_selected = selector.transform(X_resampled)
eval_X_balanced_selected = selector.transform(eval_X_balanced)

train_preds = selector.predict(X_resampled)
recall = recall_score(y_resampled, train_preds)
precision = precision_score(y_resampled, train_preds)
print('RF train', recall, precision)

eval_balanced_y_pred = selector.predict(eval_X_balanced)
recall = recall_score(eval_y_balanced, eval_balanced_y_pred)
precision = precision_score(eval_y_balanced, eval_balanced_y_pred)
print('RF test', recall, precision)

eval_y_pred = selector.predict(eval_scaled)
recall = recall_score(eval_y, eval_y_pred)
precision = precision_score(eval_y, eval_y_pred)

print('RF eval', recall, precision)


In [ ]:
SVC_model = SVC(
    kernel ='linear',
    cache_size=1000,
    random_state=42)

SVC_model.fit(X_resampled_selected, y_resampled)


In [ ]:
SVC_model_2 = SVC(
    kernel ='sigmoid',
    random_state=42)

SVC_model_2.fit(X_resampled_selected, y_resampled)

train_preds = SVC_model_2.predict(X_resampled_selected)
recall = recall_score(y_resampled, train_preds)
precision = precision_score(y_resampled, train_preds)
print('SVC2 train', recall, precision)

eval_balanced_y_pred = SVC_model_2.predict(eval_X_balanced_selected)
recall = recall_score(eval_y_balanced, eval_balanced_y_pred)
precision = precision_score(eval_y_balanced, eval_balanced_y_pred)
print('SVC2 test', recall, precision)

eval_y_pred = SVC_model_2.predict(eval_selected_scaled)
recall = recall_score(eval_y, eval_y_pred)
precision = precision_score(eval_y, eval_y_pred)
print('SVC2 eval', recall, precision)

In [ ]:

train_preds = SVC_model.predict(X_resampled_selected)
recall = recall_score(y_resampled, train_preds)
precision = precision_score(y_resampled, train_preds)
print('SVC train', recall, precision)

eval_balanced_y_pred = SVC_model.predict(eval_X_balanced_selected)
recall = recall_score(eval_y_balanced, eval_balanced_y_pred)
precision = precision_score(eval_y_balanced, eval_balanced_y_pred)
print('SVC test', recall, precision)

eval_y_pred = SVC_model.predict(eval_selected_scaled)
recall = recall_score(eval_y, eval_y_pred)
precision = precision_score(eval_y, eval_y_pred)
print('SVC test', recall, precision)

In [ ]:
LR_model = LogisticRegression(
    penalty ='l2',
    max_iter = 20000,
    random_state=42)

LR_model.fit(X_resampled_selected, y_resampled)


In [ ]:
train_preds = LR_model.predict(X_resampled_selected)
recall = recall_score(y_resampled, train_preds)
precision = precision_score(y_resampled, train_preds)
print('LR train', recall, precision)


eval_balanced_y_pred = LR_model.predict(eval_X_balanced_selected)
recall = recall_score(eval_y_balanced, eval_balanced_y_pred)
precision = precision_score(eval_y_balanced, eval_balanced_y_pred)
print('LR test', recall, precision)

eval_y_pred = LR_model.predict(eval_selected_scaled)
recall = recall_score(eval_y, eval_y_pred)
precision = precision_score(eval_y, eval_y_pred)
print('LR eval', recall, precision)

# eval_y_pred = LR_model.predict(eval_selected_scaled)
# recall = recall_score(eval_y, eval_y_pred)
# precision = precision_score(eval_y, eval_y_pred)
# print('LR', recall, precision)

In [ ]:
print(len(eval_selected_scaled)) #3308
# (3308, 161) 45 => 

# 3076 commits ??

In [ ]:
eval_y_pred4 = selector.predict(eval_scaled)
eval_y_pred = [False]*len(eval_y_pred4)

for x in range(len(eval_y_pred4)):
    if eval_y_pred4[x]:
        eval_y_pred[x]=True

In [ ]:
x_true = [False]*len(eval_y)
eval_y2 = eval_y.reset_index()
for x in range(len(eval_y2)):
    if eval_y_pred4[x] and eval_y2.iloc[x]['label_security_related']:
        x_true[x]=True
    
true_positives = sum(1  for x in x_true if x)
al_positives = (eval_df['label_security_related']==True).sum()
print(true_positives/al_positives) 


In [ ]:
import json

ix = 0
shas = []
for i, x in eval_df.iterrows():
    shas.append({
        'label_sha':x['label_sha'],
        'label_repo_full_name':x['label_repo_full_name'],
        'classification_pred': eval_y_pred[ix]
    })

    ix += 1

with open('classificated_commits.json', 'w') as f:
    json.dump(shas, f)


In [ ]:
from joblib import dump, load

dump(LR_model, 'LR_model_all.joblib') 
dump(SVC_model_2, 'SVC_model_2_all.joblib') 
dump(SVC_model, 'SVC_model_all.joblib') 
dump(LR_model, 'LR_model_all.joblib') 

